## 카카오와 네이버 통일된 최종 csv 만들기

### 데이터 불러오기

In [35]:
import pandas as pd
import numpy as np
import re

In [36]:
df_naver = pd.read_csv('data/naver_webtoon_final.csv')
df_kakao_score = pd.read_csv('data/kakao_webtoon_scoring.csv')
df_kakao = pd.read_csv('data/kakao_webtoon.csv')

### 데이터 전처리 하기

**카카오와 네이버 칼럼 규칙**  

제목 -> title  
작가(작가와 그림작가 합치기) -> artist  
장르 -> genre  
평점 -> score  
줄거리 -> story  
대표이미지 -> image  
회차별 썸네일 -> thumbs  
어느 플랫폼인지 -> from

**네이버 전처리**

In [37]:
# 필요 없는 칼럼 드랍하기

df_naver.drop('Link', axis=1, inplace=True)

In [38]:
# 칼럼 통일을 네이버 칼럼 이름 변경하기

df_naver.rename(columns = {'Title': 'title', 'Artist': 'artist', 'Genre': 'genre', 'Score(recent 10)': 'score', 'Story':'story', 'Image': 'image', "Thumbs": 'thumbs'}, inplace=True)

**카카오 전처리**

In [39]:
# 필요 없는 칼럼 드랍하기

df_kakao.drop(["id","original_story","keywords_1", "keywords_2", "keywords_3", "keywords_4","adult", "like", "view", "status"],
              axis=1, inplace=True)

In [40]:
# 필요 없는 칼럼 드랍하기

df_kakao_score.drop(['view_diff', 'like_diff', 'id'], axis=1, inplace=True)

In [41]:
df_kakao = pd.merge(df_kakao,df_kakao_score, on='title', how='outer')

In [42]:
# 네이버의 작가 형식과 통일되도록 카카오의 작가 이름 파생변수를 만들기

artist=[]
for i in range(len(df_kakao)):
    if df_kakao['writer'][i] == df_kakao['illustrator'][i]:
        artist.append(df_kakao['writer'][i])
    else:
        artist.append(df_kakao['writer'][i] + "/" + df_kakao['illustrator'][i])

df_kakao['artist'] = artist
df_kakao.drop(['writer','illustrator'], axis=1, inplace=True)

In [43]:
# 칼럼 통일을 카카오 칼럼 이름 변경하기

df_kakao.rename(columns = {'synopsis': 'story', 'timg': 'thumbs', 'thumbnail': 'image'}, inplace=True)

**함수 만들기**

In [44]:
# 각 데이터별 변수 순서 통일하기

df_naver = df_naver.reindex(columns=['title', 'artist', 'genre', 'score', 'story', 'image', 'thumbs'])
df_kakao = df_kakao.reindex(columns=['title', 'artist', 'genre', 'score', 'story', 'image', 'thumbs'])

In [45]:
# 네이버와 카카오에 각각 파생 변수 생성하기

df_naver["from"]="naver"
df_kakao["from"]="kakao"

In [46]:
# 네이버 장르를 통일된 장르로 변환하는 함수 만들기

def naver_genre_change(df):
    # 개그 -> 코믹, 스포츠 -> 학원, 무협/사극 -> 무협, 감성 -> 드라마 로 변경
    df.genre = df.genre.str.replace('개그', '코믹')
    df.genre = df.genre.str.replace('스포츠', '학원')
    df.genre = df.genre.str.replace('무협/사극', '무협')
    df.genre = df.genre.str.replace('감성', '드라마')
    df.genre
    # 장르들을 list에 넣어줌
    df.genre = df.genre.str.split(', ')
    # 스토리, 옴니버스, 에피소드 장르 삭제
    rm_set = ['스토리', '옴니버스', '에피소드']
    new_list = []
    for i in df.genre:
        new_list.append([a for a in i if a not in rm_set])
    df.genre = new_list
    return df

In [47]:
# 카카오 장르를 통일된 장르로 변환하는 함수 만들기

def kakao_genre_change(df):
    # 장르 양 옆에 있는 [' '] 삭제
    df.genre = df.genre.str.strip('['']''\'')
    # '공포/스릴러'를 '스릴러'로 변경
    df['genre'][df.genre == '공포/스릴러'] = '스릴러'
    # 장르 여러개인 경우를 각각의 장르로 분류하기 위해 장르를 list로 변환해서 넣어줌
    df.genre = df.genre.str.replace('/', ' ').str.split()
    return df

In [48]:
# 네이버와 카카오의 각 회차별 데이터에서 원하는 뽑아올 수 있는 원하는 csv 형태로 만들기

def make_thumbs_csv(df_naver, df_kakao):
    df_naver.drop_duplicates('title', inplace=True)
    df_kakao.drop_duplicates('title', inplace=True)
    
    df_kakao["thumbs"] = df_kakao["thumbs"].map(lambda x: re.sub('\[|\]|\'','',x))
    df_kakao["image"] = df_kakao["image"].map(lambda x: re.sub('\[|\]|\'','',x))
    
    df_kakao = df_kakao.drop(columns = ['image'])
    split = df_kakao.thumbs.str.split(',')
    split = split.apply(lambda x: pd.Series(x))
    split = split.stack().reset_index(level = 1, drop = True).to_frame("thumbs")
    df_kakao = df_kakao.merge(split, left_index=True, right_index=True, how ="left")
    
    df_naver = df_naver.drop(columns = ['image'])
    split = df_naver.thumbs.str.split('π')
    split = split.apply(lambda x: pd.Series(x))
    split = split.stack().reset_index(level = 1, drop = True).to_frame("thumbs")
    df_naver = df_naver.merge(split, left_index=True, right_index=True, how ="left")
    
    df_naver.drop(['thumbs_x'], axis=1, inplace=True)
    df_kakao.drop(['thumbs_x'], axis=1, inplace=True)
    
    df_naver.rename(columns={'thumbs_y': 'thumbs'}, inplace=True)
    df_kakao.rename(columns={'thumbs_y': 'thumbs'}, inplace=True)
    
    df_naver = df_naver.reindex(columns=['title', 'artist', 'genre', 'score', 'story', 'thumbs', 'from'])
    df_kakao = df_kakao.reindex(columns=['title', 'artist', 'genre', 'score', 'story', 'thumbs', 'from'])
    
    df_for_thumbs = pd.concat([df_kakao[['title', 'thumbs', 'from']], df_naver[['title', 'thumbs', 'from']]])
    
    df_for_thumbs.to_csv("webtoon_thumbs.csv", index=False)

In [49]:
make_thumbs_csv(df_naver, df_kakao)
naver_genre_change(df_naver)
kakao_genre_change(df_kakao)

C:\Users\Chalie\AppData\Local\Temp\ipykernel_3572\1415369708.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['genre'][df.genre == '공포/스릴러'] = '스릴러'


,title,artist,genre,score,story,image,thumbs,from
0,미생,윤태호,[드라마],9.976667,자신의 삶을 승리하기 위해 한 수 한 수 돌을 잇는 사람들의 이야기,https://kr-a.kakaopagecdn.com/P/C/818/sharing/...,https://kr-a.kakaopagecdn.com/P/EO/818/173349/...,kakao
1,바니와 오빠들,니은,[로맨스],9.983333,"외모 따지지 않는 '개념녀'? 아니, 나 완전 '얼빠'야! 신입생 시절, 남자의 ...",https://kr-a.kakaopagecdn.com/P/C/1781/sharing...,https://kr-a.kakaopagecdn.com/P/EO/1781/173355...,kakao
2,무지개다리 파수꾼,이서,"[판타지, 드라마]",9.976667,"돈과 성공만을 전부로 알던 유명 수의사 이한철. 그는 어느 날 사고 이후, 동물들...",https://kr-a.kakaopagecdn.com/P/C/2043/sharing...,https://kr-a.kakaopagecdn.com/P/EO/2043/173368...,kakao
3,뱀파이어님의 대리식사,멜리/웬즈데이,[로맨스],9.973333,"""대신 먹어주기만 하면 되는 '입주 대리식사' 알바?!"" 가난한 대학생 강시영은 ...",https://kr-a.kakaopagecdn.com/P/C/1959/sharing...,https://kr-a.kakaopagecdn.com/P/EO/1959/173361...,kakao
4,대사형 선유,노경찬/박창환,"[액션, 무협]",9.976667,삼류 낭인 곽노의 손에 거둬진 아이들. 사제관계를 맺고 아이들은 어설프게나마 무공...,https://kr-a.kakaopagecdn.com/P/C/2589/sharing...,https://kr-a.kakaopagecdn.com/P/EO/2589/173353...,kakao
...,...,...,...,...,...,...,...,...
1421,진심이 닿다,앤트스튜디오,[로맨스],9.776667,"자타공인 대한민국 최고 톱스타, 오윤서(본명 : 오진심!) 한창 잘나가던 그때,...",https://kr-a.kakaopagecdn.com/P/C/2082/sharing...,https://kr-a.kakaopagecdn.com/P/EO/2082/101860...,kakao
1422,드림사이드,홍정훈/신월,[스릴러],9.836667,"시골의사의 아들 남하진, 어느날 좀비가 들끓는 악몽을 꾸고 난 후, 꿈에서 만난 소...",https://kr-a.kakaopagecdn.com/P/C/1396/sharing...,https://kr-a.kakaopagecdn.com/P/EO/1396/93539/...,kakao
1423,김 비서가 왜 그럴까,김명미,[로맨스],9.933333,"천상천하유아독존, 신이 빚어낸 마스터피스, 이 세상 내 맘대로 안 되는 건 없는 유...",https://kr-a.kakaopagecdn.com/P/C/2307/sharing...,https://kr-a.kakaopagecdn.com/P/EO/2307/122198...,kakao
1424,더스크 하울러,크루/태선,"[학원, 판타지]",9.933333,소심한 성격을 극복하기 위해 밸런스 막장으로 소문난 게임 '트리키아'에 뛰어들었다!...,https://kr-a.kakaopagecdn.com/P/C/1399/sharing...,https://kr-a.kakaopagecdn.com/P/EO/1399/46249/...,kakao


In [50]:
# 최종적으로 사용할 csv로 만들기

df_total = pd.concat([df_naver,df_kakao])
df_total.reset_index(drop=True, inplace=True)
df_total.drop('thumbs', axis=1, inplace=True)
df_total.drop_duplicates('title', inplace=True)
df_total.to_csv("webtoon_total_final.csv", index=False)